In [1]:
from sqlalchemy import create_engine

engine = create_engine(
    "db2+ibm_db://daniel:db2dani@52.211.123.34:25010/IEMASTER"
)

In [3]:
import pandas as pd
from sqlalchemy import inspect

# Option 1: Using SQLAlchemy Inspector (Database Agnostic)
# This is the recommended way to explore database structure with SQLAlchemy
inspector = inspect(engine)

# List all schemas
schemas = inspector.get_schema_names()
print(f"Found {len(schemas)} schemas. First 5: {schemas[:5]}")

# List tables in the default schema
# You can pass a schema name to get_table_names(schema='SOME_SCHEMA')
tables = inspector.get_table_names()
print(f"Tables in default schema: {tables}")

# Option 2: Querying DB2 System Catalog directly (DB2 Specific)
# SYSCAT.TABLES contains metadata about tables
# We filter out system schemas (starting with SYS) to see user tables
catalog_query = """
SELECT TABSCHEMA, TABNAME, TYPE, CARD as ROW_COUNT
FROM SYSCAT.TABLES
WHERE TABSCHEMA NOT LIKE 'SYS%'
FETCH FIRST 10 ROWS ONLY
"""

pd.read_sql(catalog_query, engine)

Found 5 schemas. First 5: ['db2admin', 'ieplane', 'localadmin', 'nullid', 'sqlj']
Tables in default schema: []


,tabschema,tabname,TYPE,row_count
0,IEPLANE,COUNTRIES,T,196
1,IEPLANE,DEPARTMENT,T,22
2,IEPLANE,EMPLOYEE,T,1598
3,IEPLANE,AIRPLANES,T,120
4,IEPLANE,AIRPORTS,T,30
5,IEPLANE,ROUTES,T,59
6,IEPLANE,FLIGHTS,T,297471
7,IEPLANE,PASSENGERS,T,50000
8,IEPLANE,TICKETS,T,35383337


In [8]:
# List of tables identified in the IEPLANE schema
tables_to_check = [
    "COUNTRIES", 
    "DEPARTMENT", 
    "EMPLOYEE", 
    "AIRPLANES", 
    "AIRPORTS", 
    "ROUTES", 
    "FLIGHTS", 
    "PASSENGERS", 
    "TICKETS"
]
schema = "IEPLANE"

for table in tables_to_check:
    # Fetch first 3 rows for each table to inspect structure
    query = f"SELECT * FROM {schema}.{table} LIMIT 3"
    try:
        print(f"--- {query} ---")
        df = pd.read_sql(query, engine)
        print(df)
    except Exception as e:
        print(f"Error querying {table}: {e}")
    print("\n")

--- SELECT * FROM IEPLANE.COUNTRIES LIMIT 3 ---
          name                                   capital  population continent
0  Afghanistan  Kabul                                          67546      Asia
1      Albania  Tirana                                         95095    Europe
2      Algeria  Algiers                                        23120    Africa


--- SELECT * FROM IEPLANE.DEPARTMENT LIMIT 3 ---
  deptno                       deptname      budget       location
0    A00  COMPUTER SERVICES DIVISION HQ   629633.18  WASHINGTON DC
1    A01            OPERATIONAL SYSTEMS  6326868.78  NEW YORK CITY
2    A02         DATA WAREHOUSE SYSTEMS  6052910.64  NEW YORK CITY


--- SELECT * FROM IEPLANE.EMPLOYEE LIMIT 3 ---
      empno  firstnme midinit lastname  is_external workdept  \
0  838N0183  Kendrick       R   Craven        False      G02   
1  838N0001    Sawyer       E   Mosley        False      D02   
2  838N0002    Sophie       I    Croft        False      D01   

           